In [60]:
''' This model uses the Spacy NER package. It looks for entities within the tweets. The model is trained to find the selected text text. ''' 
!pip3 install 

ERROR: Directory 'transformers_pretrained_distilbert/' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [36]:
def Jaccard(str1, str2): 
    ## Jaccard score function, given by kaggle competition main page 

    a = set(str1.lower().split()) 
    b = set(str2.lower().split()) 
    c = a.intersection(b) 
    return float(len(c)) / (len(a) + len(b) - len(c)) 

In [37]:

def ModelOut(sentiment): 
    ## Returns the output path for the model 

    
    model_out_path = 'models/model_{}'.format(sentiment)

    return model_out_path 

positive_path = ModelOut('positive') 
negative_path = ModelOut('negative') 


In [38]:
def Train(train_data, output_dir, n_iter=20, model=None):
    ## This function was copied from Mr_KnowNothing. 
    ## https://www.kaggle.com/rohitsingh9990/ner-training-using-spacy-ensemble
    ## It creates the training model for the ner, and outputs it to the directory 
    ## I found this function being used in numerous notebooks to train the data 
    ## This code was adapted from the Spacy github repo. 
    ## https://github.com/explosion/spacy/blob/master/examples/training/train_ner.py



    if model is not None:
        nlp = spacy.load(output_dir)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
        if model is None:
            nlp.begin_training()
        else:
            nlp.resume_training()

        for itn in tqdm(range(n_iter)):
            random.shuffle(train_data)
            batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,   # dropout - make it harder to memorise data
                    losses=losses, 
                )
            
            print("Losses", losses)

    
    ## Output to the working directory 
    output_dir = '{}/{}'.format(os.getcwd(), output_dir)
    print(output_dir) 

    
    nlp.meta['name'] = 'st_ner' 
    ## put the model to disk 
    nlp.to_disk(output_dir) 

    print('saved model to {}'.format(output_dir) ) 



In [51]:
import pandas as pd 
import numpy as np
import spacy
from pathlib import Path
import os
from tqdm import tqdm
import random
from spacy.util import minibatch, compounding
from spacy.gold import GoldParse
import transformers 
import tokenizers
filepath = os.getcwd().replace('finalproject',  '')

## Read in the test and the train data
train = pd.read_csv('train.csv') 
test = pd.read_csv('test.csv') 

In [52]:
def SaveModel(directory, nlp, model_name): 
    ## Save the NER model to the directory 

    directory = filepath + directory 

    if directory is not None: 
        os.makedirs(directory) 
    
    nlp.meta['name'] = model_name 
    nlp.to_disk(directory) 
    print('saved model to {}'.format(directory) )

In [53]:
##It was found that words with the word count 3 or less equalled the total tweet. So for this model, we are going to take them out. 
train['word_count'] = train['text'].apply(lambda x: len(str(x).split(' ')))
train = train[train.word_count >= 3]

## Drop na as well 
train = train.dropna() 


In [54]:
## For the Spacy NER Model, we need the start and end locations of the selected text within the full tweet 

train['first_index'] = train.apply(lambda x: x['text'].find(x['selected_text']), axis = 1)  

train['last_index'] = train.selected_text.str.len() + train.first_index

In [55]:
## Spacy NER appropriate dictionary format 
train['spacy_format'] = train.apply(lambda x: {'entities': [[x['first_index'], x['last_index'], 'selected_text']]}, axis = 1)

In [56]:
## Split the data into positve, and negative sentiments

def SentimentSplit(df): 
    ## Split the dataframe by sentiment into a dictionary
    

    positive_sentiment = df[df.sentiment == 'positive'] 
    negative_sentiment = df[df.sentiment == 'negative']

    sentiment = {'positive': positive_sentiment, 
                    'negative': negative_sentiment}

    return sentiment


train_sentiment = SentimentSplit(train) 
test_sentiment = SentimentSplit(test) 

train_sentiment['negative'].head(1)


,textID,text,selected_text,sentiment,word_count,first_index,last_index,spacy_format
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,11,1,9,"{'entities': [[1, 9, 'selected_text']]}"


In [57]:
def SentimentSplit(train_sentiment): 
    ## convert the train data to a list, properly formatted for the Spacy. 

    train_data = [] 
    for index, row in train_sentiment.iterrows(): 
        spacy_format = {'entities': [[row.first_index, row.last_index, 'selected_text']]}
        train_data.append((row.text, spacy_format) )

    return train_data 

positive_sentiment = SentimentSplit(train_sentiment['positive']) 
negative_sentiment = SentimentSplit(train_sentiment['negative'])

In [58]:
## Train the model using Spacy NER. The function used was adapted from the Spacy github, and was consistently used through plenty of notebooks within the kaggle competition. 

Train(positive_sentiment, positive_path, n_iter = 2, model = None)

Train(negative_sentiment, negative_path, n_iter = 2, model = None) 

ImportError: cannot import name 'PreTrainedTokenizerFast' from 'transformers.tokenization_utils' (/Users/vijayyedidi/opt/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils.py)

In [ ]:
## Load in the model 
positive_model = spacy.load(os.getcwd() + '/models/model_positive')
negative_model = spacy.load(os.getcwd() + '/models/model_positive') 


def Prediction(text, model): 

    ## This function was also taken from a kaggle notebook. Ricardo Cantu Martinez 
    ## https://www.kaggle.com/ricardocantumartinez had this code to properly interpret the model. 
    ## Without extracting the start and end points of the data, and then finding the substring within the 
    ## text that corresponded to the entity selected, the model was skipping over rows and not making complete predictions. 
    
    doc = model(text) 
    ent_array = [] 
    for ent in doc.ents: 
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text

In [ ]:


def Output(data): 
    selected_text = []
 ## Creates final submission column
    for index, row in data.iterrows(): 
        text = row.text 

        ## Output the total text for neutral tweets and for tweets less than 3 words long
        if row.sentiment == 'neutral' or len(text.split(' '))<= 2: 
            selected_text.append(text) 
        
        ## Output the total text for positive tweets 
        elif row.sentiment == 'positive': 
            selected_text.append(Prediction(text, positive_model) ) 

        ## Output the total text for negative tweets 
        elif row.sentiment == 'negative': 
            selected_text.append(Prediction(text, negative_model))

    return selected_text

test['selected_text'] = Output(test) 
train['output'] = Output(train) 



In [ ]:
jaccard = [] 

for index, row in train.iterrows(): 

    ## Calculate the final jaccard score for the training data

    selected_text = row.selected_text 
    output = row.output 

    jaccard.append(Jaccard(selected_text, output) ) 

train['jaccard'] =  jaccard

In [ ]:
## We want to see the total jaccard score when running the model on the training data. It comes out to 0.618354. 
total_jaccard = train.jaccard.mean()
print(total_jaccard) 

In [ ]:
## Create the submission file
submission = test[['textID', 'selected_text']]
submission.to_csv('NER_Submission.csv', index = False) 